# shell

## Recipes

### To navigate to the previous directory (or back)

```sh
cd -
```

### Expanding commands
```sh
echo "There are $(ls | wc -w) files in this directory."
```

### Shell variables

```sh
echo $PATH
```

### Write to file

```sh
cat sometext > newfile.txt
```

### Append to file

```sh
cat sometext >> existing_or_newfile.txt
```

### Write command output to file and redirect errors to stdout

```sh
cmd > file.txt 2>&1
```

### Find files by name 

```sh
find /etc name '*passwd*'
```

### Find files by name case insensitive

```sh
find /etc -iname '*passwd*'
```


### Find files and execute

```sh
find /etc -iname iptables -exec echo "I found {}" \;
```

### Find and separate files with a 0 (NULL) byte so that names containing spaces or newlines can be interpreted correctly

```sh
find . -type f -name '*gz' -print0
```

### grep case insensitive keyword in a file

```sh
grep -i keyword /path/to/file
```

### grep recursively and list only files

```sh
grep -rl keyword /usr/share/doc/
```

### grep on command output

```sh
ip addr show | grep keyword
```

### list files matching regex pattern

```sh
ls [agw]*[ne]
```

### xargs delete all zip files

```sh
find . -type f -name '*gz' -print0 | xargs -0 -I{} rm {}
```

### xargs execute command on whole result

```sh
find . -name '*.txt' -print0 | xargs -0 
```

### xargs execute command per line

```sh
find . -name '*.txt' -print0 | xargs -0 -n 1
```

If there is no space or new line in file names, this works too:

```sh

ls -1 | xargs -L1 wc -l
find . -name '*.txt' | xargs -L1 echo 'Path: '

```

### Move All files in a folder to another folder

```sh
find . -type f -name '*' -print0 | xargs -0 mv -t ..
```

### Concatenate string by xargs

```sh
cat file.txt | tr '\n' '\0' | xargs -0 -L1 -I{} echo 'prefix{}'
```

### Compress 

```sh
tar -cvf archive.tar directory_or_file
zip myzip file1 file2 file3`
```

### Decompress (extract) a zip file

```sh
tar -xvf archive.tar directory
unzip Myzip.zip
```

### Find command

```sh
which command
```

### Create alias

```sh
alias p='pwd ; ls –CF'
```

### Make file executable

```sh
chmod 700 file
```

### Create multiple numbered files

```sh
touch memo{1,2,3,4,5}
```

### Kill port process

```sh
lsof -i :8000
kill -9 <PID>
```

### Mount volume 

```sh
sudo mount /dev/xvdf /data
```

### Unmount volume

```sh
sudo umount /dev/xvdf
```

### Place a program in the background by putting an ampersand (&) at the end

```sh
find /usr > /tmp/allusrfiles &
```

### Separate by a delimiter and get 6th field

```sh
echo $PATH | cut -d':' -f6
```

### Replace spaces in filenames with underscores

```sh
for file in * ; do
	f=`echo $file | tr [:blank:] [_]`
	[ "$file" = "$f" ] || mv -i -- "$file" "$f"
done
```

### Send process background

```
<Ctrl + z>
bg
disown %1
```

### Parse json and get value

```sh

function jsonValue {
	KEY=$1
	num=$2
	awk -F"[,:}]" '{for(i=1;i<=NF;i++){if($i~/'$KEY'\042/){print $(i+1)}}}' | tr -d '"' | sed -n ${num}p
}
curl -s -X GET http://twitter.com/users/show/$1.json | jsonValue profile_image_url 1

```

### Pass string when file expected

```sh

# <(CMD)
diff <(ls foo) <(ls bar)

```

### Show date as UTC timestamp

```sh

date +%s

```

### Add a prefix to every file in the directory

```sh

for f in * ; do mv -- "$f" "2022-02-04-$f" ; done

# or
for f in $(find . -maxdepth 1 -type f  -name '*.json'); do  
mv -v $f ${f%/*}/$(date +%Y-%m-%d)_prefix_${f##*/}
done

```

### Format date to ISO format

```sh
date +%Y-%m-%d-T%H:%M:%S%z
```

### Export environment variables from .env file 

```sh
export $(echo $(cat .env | sed 's/#.*//g' | sed 's/\r//g' | xargs) | envsubst)
```

[reference](https://gist.github.com/mihow/9c7f559807069a03e302605691f85572?permalink_comment_id=3700180#gistcomment-3700180)


### Command line arguments with varargs and flags

```sh

# usage
# cli.sh -l Newton EINSTEIN HawKING
# output: newton einstein hawking
lower=1
while getopts ":l:u:" flag
do
	case "${flag}" in
		l) 
		lower=1
		OPTIND=$OPTIND-1
		;;
		u) lower=0
		OPTIND=$OPTIND-1
		;;
		\?)
		echo "Invalid option: -$OPTARG" >&2
		exit 1
		;;
		:)
		echo "Option -$OPTARG requires an argument." >&2
		exit 1
		;;
	esac
done
shift $((OPTIND -1))

for word in $@
do
if [[ $lower -eq 1 ]]; then
	echo -n $word ' '| tr '[:upper:]' '[:lower:]'
else
	echo -n $word ' '| tr '[:lower:]' '[:upper:]'
fi
done
echo 

```

## Bash zine summary

- Shebang 

	```sh

	!#/bin/bash

	```

- Check shell script for errors 

	```sh

	shellcheck my-script.sh

	```

- Always escape variables with double quote for expansion

	```sh

	filename="my file.txt"

	echo "$filename"

	echo "${filename}.zip" # to concatenate

	```

- See all environment variables with `env`

- Use single quotes to not expand string, i.e. take it exactly same

- Bash automatically expands globs (`*`) if not quoted

- Bash skips hidden files when expanding globs

- sudo does not affect redirects 

	```sh

	echo "something" > sudo tee /etc/xyz

	```

- Always use double brackets for if conditions

- Use `local` for local variables

	```sh

	local x
	x=$(ls -l)
	# this one never fails
	local y=$(some_error_prone_command)

	```

- Defining functions

	```sh

	say_hello() {
		# all arguments $@
		# script name $0
		# arguments are $1, $2 ...
		echo "Hello $1 $2"
	}

	```

- Concurrent processes

    - Wait for multiple commands to finish. Each terminal window has its own set of jobs.

    ```sh

    command1 &
    command2 &
    wait

    ```

- Parameter expansion `${...}`

    - length of string or array `${#var}`

    - remove prefix/suffix

        ```sh

        ${var#pattern} # prefix
        ${var%pattern} # suffix

        ```

    - substring `${var:offset:length}`

- `trap` command. It's like `defer` in Golang.

    - Syntax `trap COMMAND EVENT`

		```sh

		function clean_up() {
			rm -rf $TEMP_DIR
			rm $TEMP_FILE
		} 
		trap cleanup EXIT

		```



- Errors

    - Stop execution on errors, undefined variables and fail pipe

		```sh

		set -euo pipefail

		```

    - Print error message after failure

		```sh

		die() { echo $1 >%2; exit 1; }
		some_command || die 'oh no'

		```

- `set -x` prints each line as executed

